In [1]:
library(tidyverse)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "SQL")
list.files(sql_dir)

[1] "cohort.sql"      "data.sql"        "lab_tests.sql"   "population.sql" 
[5] "vital_signs.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic", user = "lcp_db_user")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "vital_signs.sql")) %>%
    make_view_sql("vital_signs", "view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from vital_signs limit 10;
"
dbGetQuery(con, sql)

subject_id,hadm_id,icustay_id,charttime,label,valuenum
250,124271,205951,2188-11-13 06:30:00,map,73
250,124271,205951,2188-11-13 06:30:00,cvp,18
250,124271,205951,2188-11-13 06:45:00,map,73
250,124271,205951,2188-11-13 06:45:00,cvp,18
250,124271,205951,2188-11-13 07:00:00,map,73
250,124271,205951,2188-11-13 07:00:00,cvp,19
252,190159,221239,2133-03-31 12:00:00,map,91
252,190159,221239,2133-04-01 04:00:00,map,69
252,190159,221239,2133-04-01 04:00:00,cvp,12
250,124271,205951,2188-11-13 04:15:00,map,70


In [8]:
sql <- readr::read_file(file.path(sql_dir, "lab_tests.sql")) %>%
    make_view_sql("lab_tests", "view")

In [9]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [10]:
sql <- "
select * from lab_tests limit 10;
"
dbGetQuery(con, sql)

subject_id,hadm_id,icustay_id,charttime,label,valuenum
3,145834,211552,2101-10-20 19:26:00,bun,41.0
3,145834,211552,2101-10-20 19:26:00,hemoglobin,7.8
3,145834,211552,2101-10-20 19:26:00,platelet,190.0
3,145834,211552,2101-10-20 19:26:00,wbc,11.3
3,145834,211552,2101-10-20 19:59:00,bicarbonate,11.0
3,145834,211552,2101-10-20 19:59:00,chloride,111.0
3,145834,211552,2101-10-20 19:59:00,creatinine_kinase,82.0
3,145834,211552,2101-10-20 19:59:00,creatinine_kinase,NA
3,145834,211552,2101-10-20 19:59:00,creatinine,2.5
3,145834,211552,2101-10-20 19:59:00,potassium,4.1


In [11]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE